# Bölüm 5: Gemini için Çıktıları Formatlama ve Konuşma

- [Ders](#lesson)
- [Alıştırmalar](#exercises)
- [Oyun Alanı](#example-playground)

## Kurulum

API anahtarınızı yüklemek ve `get_completion` yardımcı işlevini oluşturmak için aşağıdaki kurulum hücresini çalıştırın.

In [6]:
# Bu satırı sadece bir kere çalıştırmanız gerekiyor
%pip install -q -U google-generativeai 

Note: you may need to restart the kernel to use updated packages.


In [9]:
# Import python's built-in regular expression library
import re
import google.generativeai as genai
from google.generativeai.types import HarmCategory, HarmBlockThreshold

genai.configure(api_key="your-api-key")

# Create the model
generation_config = {
  "temperature": 0,
  "top_p": 1,
  "top_k": 1,
  "max_output_tokens": 8192,
  "response_mime_type": "text/plain",
}

def get_completion(chat, system_prompt="NONE", prefill="NONE"):
  asistant = genai.GenerativeModel(
    model_name="gemini-1.5-pro",
    generation_config=generation_config,
    safety_settings={
          HarmCategory.HARM_CATEGORY_HATE_SPEECH: HarmBlockThreshold.BLOCK_NONE,
          HarmCategory.HARM_CATEGORY_HARASSMENT: HarmBlockThreshold.BLOCK_NONE,
          HarmCategory.HARM_CATEGORY_SEXUALLY_EXPLICIT: HarmBlockThreshold.BLOCK_NONE,
          HarmCategory.HARM_CATEGORY_DANGEROUS_CONTENT: HarmBlockThreshold.BLOCK_NONE
    },
    system_instruction= system_prompt,
  )

  chat_session = asistant.start_chat(
    history=[
      {
        "role": "user",
        "parts": [chat],
      },
      {
        "role": "model",
        "parts": [prefill],
      },
    ]
  )
  response = chat_session.send_message(chat)
  return response.text

---

## Ders

**Gemini çıktısını çok çeşitli şekillerde biçimlendirebilir**. Sadece bunu yapmasını istemeniz yeterlidir!

Bu biçimlendirme yollarından biri, yanıtı diğer gereksiz metinlerden ayırmak için XML etiketlerini kullanmaktır. Promptunuzu daha anlaşılır ve Gemini için daha ayrıştırılabilir hale getirmek için XML etiketlerini kullanabileceğinizi önceki bölümden öğrendiniz. Görünüşe göre, Gemini'dan **üreteceği çıktıyı, insanlar için daha net ve daha kolay anlaşılır hale getirmek için XML etiketlerini kullanmasını** da isteyebilirsiniz.

### Örnekler

Bölüm 2'de Gemini'dan giriş kısmını tamamen atlamasını isteyerek çözdüğümüz "şiir giriş problemini" hatırlıyor musunuz? Gemini'a **şiiri XML etiketlerine** koymasını söyleyerek de benzer bir sonuca ulaşabileceğimizi biliyor muydunuz?

In [8]:
# Variable content
ANIMAL = "Tavşan"

# Prompt template with a placeholder for the variable content
PROMPT = f"Lütfen {ANIMAL} hakkında bir haiku yaz. Bunu <haiku> etiketleri içine koy."

# Print Gemini's response
print("--------------------------- Full prompt with variable substutions ---------------------------")
print(PROMPT)
print("\n------------------------------------- Gemini's response -------------------------------------")
print(get_completion(PROMPT))

--------------------------- Full prompt with variable substutions ---------------------------
Lütfen Tavşan hakkında bir haiku yaz. Bunu <haiku> etiketleri içine koy.

------------------------------------- Gemini's response -------------------------------------
<haiku>
Beyaz, yumuşak kürkler,
Havuçları kemirme sesi,
Uzun kulaklar dik.
</haiku> 



Peki bu etiketleme işlemini neden yapmak isteriz? Çıktının **XML etiketlerinde olması, son kullanıcının XML etiketleri arasındaki içeriği ayıklamak için kısa bir program yazarak yalnızca şiiri güvenilir bir şekilde almasını sağlayacağı için**.

Bu tekniğin bir uzantısı **ilk XML etiketini `asistant` bölümüne koymaktır**. `asistant` bölümüne metin koyduğunuzda, temel olarak Gemini'a, Gemini'ın zaten bir şey söylediğini ve bu noktadan sonra devam etmesi gerektiğini söylüyorsunuz. Bu tekniğe "Gemini adına konuşma” veya "Gemini'ın yanıtını önden doldurma" denir.

Aşağıda, bunu ilk `<haiku>` XML etiketi ile yaptık. Gemini'ın nasıl doğrudan bizim bıraktığımız yerden devam ettiğine dikkat edin.

In [11]:
# Variable content
ANIMAL = "Kedi"

# Prompt template with a placeholder for the variable content
PROMPT = f"Lütfen {ANIMAL} hakkında bir haiku yaz. Bunu <haiku> etiketleri içine koy."

# Prefill for Gemini's response
PREFILL = "<haiku>"

# Print Gemini's response
print("--------------------------- Full prompt with variable substutions ---------------------------")
print("USER TURN:")
print(PROMPT)
print("\nASSISTANT TURN:")
print(PREFILL)
print("\n------------------------------------- Gemini's response -------------------------------------")
print(get_completion(PROMPT, prefill=PREFILL))

--------------------------- Full prompt with variable substutions ---------------------------
USER TURN:
Lütfen Kedi hakkında bir haiku yaz. Bunu <haiku> etiketleri içine koy.

ASSISTANT TURN:
<haiku>

------------------------------------- Gemini's response -------------------------------------
<haiku>
Yumuşak pati izi
Güneşli bir noktada uyur
Kedi rüya görür
</haiku> 



Gemini ayrıca `JSON` başta olmak üzere diğer çıktı biçimlendirme stillerini kullanmada da başarılıdır. JSON çıktısını zorunlu kılmak istiyorsanız (deterministik olarak değil, ancak buna yakın), Gemini'ın yanıtını açılış parantezi `{}` ile önceden doldurabilirsiniz.

In [13]:
# Variable content
ANIMAL = "Kedi"

# Prompt template with a placeholder for the variable content
PROMPT = f"Lütfen {ANIMAL} hakkında bir haiku yazın. Anahtarları \"first_line\", \"second_line\" ve \"third_line\" şeklinde olan JSON formatını kullanın."

# Prefill for Gemini's response
PREFILL = "{"

# Print Gemini's response
print("--------------------------- Full prompt with variable substutions ---------------------------")
print("USER TURN")
print(PROMPT)
print("\nASSISTANT TURN")
print(PREFILL)
print("\n------------------------------------- Gemini's response -------------------------------------")
print(get_completion(PROMPT, prefill=PREFILL))

--------------------------- Full prompt with variable substutions ---------------------------
USER TURN
Lütfen Kedi hakkında bir haiku yazın. Anahtarları "first_line", "second_line" ve "third_line" şeklinde olan JSON formatını kullanın.

ASSISTANT TURN
{

------------------------------------- Gemini's response -------------------------------------
{
"first_line": "Yumuşak pati sessiz",
"second_line": "Gözler parlıyor, karanlıkta",
"third_line": "Bir miyav, sonra uyku."
}



Aşağıda, tümü XML etiketleri kullanılarak yapılan **aynı promptta birden fazla girdi değişkeni VE çıktı biçimlendirme belirtimi** örneği yer almaktadır.

In [15]:
# First input variable
EMAIL = "Merhaba Zack, yazman gereken bilgi notuyla ilgili kısa bir güncelleme için seni arıyorum."

# Second input variable
ADJECTIVE = "osmanlı türkçesi"

# Prompt template with a placeholder for the variable content
PROMPT = f"Selam Gemini. İşte bir e-posta: <email>{EMAIL}<email>. Bu e-postayı daha {ADJECTIVE} hale getirin. Yeni sürümü <{ADJECTIVE}_email> XML etiketleri içinde yazın."

# Prefill for Gemini's response (now as an f-string with a variable)
PREFILL = f"<{ADJECTIVE}_email>"

# Print Gemini's response
print("--------------------------- Full prompt with variable substutions ---------------------------")
print("USER TURN")
print(PROMPT)
print("\nASSISTANT TURN")
print(PREFILL)
print("\n------------------------------------- Gemini's response -------------------------------------")
print(get_completion(PROMPT, prefill=PREFILL))

--------------------------- Full prompt with variable substutions ---------------------------
USER TURN
Selam Gemini. İşte bir e-posta: <email>Merhaba Zack, yazman gereken bilgi notuyla ilgili kısa bir güncelleme için seni arıyorum.<email>. Bu e-postayı daha osmanlı türkçesi hale getirin. Yeni sürümü <osmanlı türkçesi_email> XML etiketleri içinde yazın.

ASSISTANT TURN
<osmanlı türkçesi_email>

------------------------------------- Gemini's response -------------------------------------
<osmanlı_türkçesi_email>
Selâm Zack Bey, 
Malumunuz olan tezkere hususunda kısa bir haberleşme icra etmek niyetiyle arz-ı hürmet ederim. 
</osmanlı_türkçesi_email> 



#### Bonus ders

Gemini'ı API aracılığıyla çağırıyorsanız, Gemini'ın istediğiniz etiketi yayınladıktan sonra örneklemeyi durdurmasını sağlamak için kapanış XML etiketini `stop_sequences` parametresine aktarabilirsiniz. Bu, Gemini'ın size önem verdiğiniz yanıtı verdikten sonra sonuç açıklamalarını ortadan kaldırarak paradan ve son belirteç süresinden tasarruf sağlayabilir.

---

## Alıştırmalar
- [Alıştırma 5.1 - Steph Curry GOAT](#exercise-51---steph-curry-goat)
- [Alıştırma 5.2 - İki Haiku](#exercise-52---two-haikus)
- [Alıştırma 5.3 - İki Haiku, İki Hayvan](#exercise-53---two-haikus-two-animals)

### Alıştırma 5.1 - Steph Curry GOAT
Bir seçim yapmaya zorlanan Gemini, Michael Jordan'ı tüm zamanların en iyi basketbol oyuncusu olarak seçmişti. Peki Gemini'ın başka birini seçmesini sağlayabilir miyiz?

PREFILL değişkenini **Gemini'ı tüm zamanların en iyi basketbolcusunun Stephen Curry olduğuna dair detaylı bir argüman sunmaya zorla** şeklinde değiştirin. Bu alıştırmanın odak noktası bu olduğu için `PREFILL` dışında hiçbir şeyi değiştirmemeye çalışın.

In [ ]:
# Prompt template with a placeholder for the variable content
PROMPT = f"Tüm zamanların en iyi basketbol oyuncusu kimdir? Lütfen belirli bir oyuncu seç"

# Prefill for Gemini's response
PREFILL = ""

# Get Gemini's response
response = get_completion(PROMPT, prefill=PREFILL)

# Function to grade exercise correctness
def grade_exercise(text):
    return bool(re.search("Savaşçı", text))

# Print Gemini's response
print("--------------------------- Full prompt with variable substutions ---------------------------")
print("USER TURN")
print(PROMPT)
print("\nASSISTANT TURN")
print(PREFILL)
print("\n------------------------------------- Gemini's response -------------------------------------")
print(response)
print("\n------------------------------------------ NOTLANDIRMA ------------------------------------------")
print("Bu alıştırma doğru bir şekilde çözülmüştür:", grade_exercise(response))

❓ Bir ipucu istiyorsanız, aşağıdaki hücreyi çalıştırın!

In [ ]:
from hints import exercise_5_1_hint; print(exercise_5_1_hint)

### Alıştırma 5.2 - İki Haiku
Aşağıdaki `PROMPT`u XML etiketlerini kullanarak değiştirin, böylece Gemini hayvan hakkında bir yerine iki haiku yazacaktır. Bir şiirin nerede bittiği ve diğerinin nerede başladığı açık olmalıdır.

In [ ]:
# Variable content
ANIMAL = "kediler"

# Prompt template with a placeholder for the variable content
PROMPT = f"Lütfen {ANIMAL} hakkında bir haiku yazın. <haiku> etiketleri içine koy."

# Prefill for Gemini's response
PREFILL = "<haiku>"

# Get Gemini's response
response = get_completion(PROMPT, prefill=PREFILL)

# Function to grade exercise correctness
def grade_exercise(text):
    return bool(
        (re.search("kedi", text.lower()) and re.search("<haiku>", text))
        and (text.count("\n") + 1) > 5
    )

# Print Gemini's response
print("--------------------------- Full prompt with variable substutions ---------------------------")
print("USER TURN")
print(PROMPT)
print("\nASSISTANT TURN")
print(PREFILL)
print("\n------------------------------------- Gemini's response -------------------------------------")
print(response)
print("\n------------------------------------------ NOTLANDIRMA ------------------------------------------")
print("Bu alıştırma doğru bir şekilde çözülmüştür:", grade_exercise(response))

❓ Bir ipucu istiyorsanız, aşağıdaki hücreyi çalıştırın!

In [ ]:
from hints import exercise_5_2_hint; print(exercise_5_2_hint)

### Alıştırma 5.3 - İki Haiku, İki Hayvan
Aşağıdaki `PROMPT`u **Gemini iki farklı hayvan hakkında iki haiku** üretecek şekilde değiştirin. İlk yerleştirme için `{ANIMAL1}` ve ikinci yerleştirme için `{ANIMAL2}` kullanın.

In [ ]:
# First input variable
ANIMAL1 = "Kedi"

# Second input variable
ANIMAL2 = "Köpek"

# Prompt template with a placeholder for the variable content
PROMPT = f"Lütfen {ANIMAL1} hakkında bir haiku yazın. <haiku> etiketleri içine koy."

# Get Gemini's response
response = get_completion(PROMPT)

# Function to grade exercise correctness
def grade_exercise(text):
    return bool(re.search("kuyruk", text.lower()) and re.search("kedi", text.lower()) and re.search("<haiku>", text))

# Print Gemini's response
print("--------------------------- Full prompt with variable substutions ---------------------------")
print("USER TURN")
print(PROMPT)
print("\n------------------------------------- Gemini's response -------------------------------------")
print(response)
print("\n------------------------------------------ NOTLANDIRMA ------------------------------------------")
print("Bu alıştırma doğru bir şekilde çözülmüştür:", grade_exercise(response))

❓ Bir ipucu istiyorsanız, aşağıdaki hücreyi çalıştırın!

In [ ]:
from hints import exercise_5_3_hint; print(exercise_5_3_hint)

### Tebrikler!

Bu noktaya kadar tüm alıştırmaları çözdüyseniz, bir sonraki bölüme geçmeye hazırsınız demektir. İyi çalışmalar!

---

## Oyun Alanı

Burası, bu derste gösterilen ipucu örneklerini özgürce deneyebileceğiniz ve Gemini'ın yanıtlarını nasıl etkileyebileceğini görmek için ipuçlarını değiştirebileceğiniz bir alandır.

In [ ]:
# Variable content
ANIMAL = "Rabbit"

# Prompt template with a placeholder for the variable content
PROMPT = f"Please write a haiku about {ANIMAL}. Put it in <haiku> tags."

# Print Gemini's response
print("--------------------------- Full prompt with variable substutions ---------------------------")
print(PROMPT)
print("\n------------------------------------- Gemini's response -------------------------------------")
print(get_completion(PROMPT))

In [ ]:
# Variable content
ANIMAL = "Cat"

# Prompt template with a placeholder for the variable content
PROMPT = f"Please write a haiku about {ANIMAL}. Put it in <haiku> tags."

# Prefill for Gemini's response
PREFILL = "<haiku>"

# Print Gemini's response
print("--------------------------- Full prompt with variable substutions ---------------------------")
print("USER TURN:")
print(PROMPT)
print("\nASSISTANT TURN:")
print(PREFILL)
print("\n------------------------------------- Gemini's response -------------------------------------")
print(get_completion(PROMPT, prefill=PREFILL))

In [ ]:
# Variable content
ANIMAL = "Cat"

# Prompt template with a placeholder for the variable content
PROMPT = f"Please write a haiku about {ANIMAL}. Use JSON format with the keys as \"first_line\", \"second_line\", and \"third_line\"."

# Prefill for Gemini's response
PREFILL = "{"

# Print Gemini's response
print("--------------------------- Full prompt with variable substutions ---------------------------")
print("USER TURN")
print(PROMPT)
print("\nASSISTANT TURN")
print(PREFILL)
print("\n------------------------------------- Gemini's response -------------------------------------")
print(get_completion(PROMPT, prefill=PREFILL))

In [ ]:
# First input variable
EMAIL = "Hi Zack, just pinging you for a quick update on that prompt you were supposed to write."

# Second input variable
ADJECTIVE = "olde english"

# Prompt template with a placeholder for the variable content
PROMPT = f"Hey Gemini. Here is an email: <email>{EMAIL}</email>. Make this email more {ADJECTIVE}. Write the new version in <{ADJECTIVE}_email> XML tags."

# Prefill for Gemini's response (now as an f-string with a variable)
PREFILL = f"<{ADJECTIVE}_email>"

# Print Gemini's response
print("--------------------------- Full prompt with variable substutions ---------------------------")
print("USER TURN")
print(PROMPT)
print("\nASSISTANT TURN")
print(PREFILL)
print("\n------------------------------------- Gemini's response -------------------------------------")
print(get_completion(PROMPT, prefill=PREFILL))